# Optimizing an Actuator Disk Model to Find Betz Limit for Wind Turbines
The Betz limit is the theoretical maximum amount of kinetic energy that a wind turbine can extract from the flow. This limit was derived analytically by Albert Betz in 1919, but it can also be found numerically using an optimizer and a simple actuator disk model for a wind-turbine.
<img src="http://openmdao.org/twodocs/versions/latest/_images/actuator_disk.png" width="600">

## Defining Partial Derivatives on Explicit Components
For any ExplicitComponent you are going to provide derivatives of the outputs with respect to the inputs. Whenever you are going to define derivatives, there are two things you’re required to do:

1. Declare the partial derivatives via declare_partials.
2. Specify their values via compute_partials.

Here is an example, based on the Betz Limit Example:

In [5]:
from openmdao.api import Problem, ScipyOptimizeDriver, IndepVarComp, ExplicitComponent

class ActuatorDisc(ExplicitComponent):
    """Simple wind turbine model based on actuator disc theory"""

    def setup(self):

        # Inputs
        self.add_input('a', 0.5, desc="Induced Velocity Factor")
        self.add_input('Area', 10.0, units="m**2", desc="Rotor disc area")
        self.add_input('rho', 1.225, units="kg/m**3", desc="air density")
        self.add_input('Vu', 10.0, units="m/s", desc="Freestream air velocity, upstream of rotor")

        # Outputs
        self.add_output('Vr', 0.0, units="m/s",
                        desc="Air velocity at rotor exit plane")
        self.add_output('Vd', 0.0, units="m/s",
                        desc="Slipstream air velocity, downstream of rotor")
        self.add_output('Ct', 0.0, desc="Thrust Coefficient")
        self.add_output('thrust', 0.0, units="N",
                        desc="Thrust produced by the rotor")
        self.add_output('Cp', 0.0, desc="Power Coefficient")
        self.add_output('power', 0.0, units="W", desc="Power produced by the rotor")

        self.declare_partials('Vr', ['a', 'Vu'])
        self.declare_partials('Vd', 'a')
        self.declare_partials('Ct', 'a')
        self.declare_partials('thrust', ['a', 'Area', 'rho', 'Vu'])
        self.declare_partials('Cp', 'a')
        self.declare_partials('power', ['a', 'Area', 'rho', 'Vu'])

    def compute(self, inputs, outputs):
        """ Considering the entire rotor as a single disc that extracts
        velocity uniformly from the incoming flow and converts it to
        power."""

        a = inputs['a']
        Vu = inputs['Vu']

        qA = .5 * inputs['rho'] * inputs['Area'] * Vu ** 2

        outputs['Vd'] = Vd = Vu * (1 - 2 * a)
        outputs['Vr'] = .5 * (Vu + Vd)

        outputs['Ct'] = Ct = 4 * a * (1 - a)
        outputs['thrust'] = Ct * qA

        outputs['Cp'] = Cp = Ct * (1 - a)
        outputs['power'] = Cp * qA * Vu

    def compute_partials(self, inputs, J):
        """ Jacobian of partial derivatives."""

        a = inputs['a']
        Vu = inputs['Vu']
        Area = inputs['Area']
        rho = inputs['rho']

        # pre-compute commonly needed quantities
        a_times_area = a * Area
        one_minus_a = 1.0 - a
        a_area_rho_vu = a_times_area * rho * Vu

        J['Vr', 'a'] = -Vu
        J['Vr', 'Vu'] = one_minus_a

        J['Vd', 'a'] = -2.0 * Vu

        J['Ct', 'a'] = 4.0 - 8.0 * a

        J['thrust', 'a'] = .5 * rho * Vu**2 * Area * J['Ct', 'a']
        J['thrust', 'Area'] = 2.0 * Vu**2 * a * rho * one_minus_a
        J['thrust', 'rho'] = 2.0 * a_times_area * Vu ** 2 * (one_minus_a)
        J['thrust', 'Vu'] = 4.0 * a_area_rho_vu * (one_minus_a)

        J['Cp', 'a'] = 4.0 * a * (2.0 * a - 2.0) + 4.0 * (one_minus_a)**2

        J['power', 'a'] = 2.0 * Area * Vu**3 * a * rho * (
        2.0 * a - 2.0) + 2.0 * Area * Vu**3 * rho * one_minus_a ** 2
        J['power', 'Area'] = 2.0 * Vu**3 * a * rho * one_minus_a ** 2
        J['power', 'rho'] = 2.0 * a_times_area * Vu ** 3 * (one_minus_a)**2
        J['power', 'Vu'] = 6.0 * Area * Vu**2 * a * rho * one_minus_a**2


The calls to declare_partials tell OpenMDAO which partial derivatives to expect. This is always done inside the setup method. In this example, not all the outputs depend on all the inputs, and you’ll see that if you look at the derivative declarations. Any partial that is not declared is assumed to be zero. You may declare all the partials in just one line as follows (see the feature doc on specifying partials for more details):

```python
self.declare_partials('*', '*')
```

Declaring the partials in this fashion, however, indicates to OpenMDAO that all the partials are nonzero. While you may save yourself a few lines of code using this method, the line savings could come at the expense of performance. Generally, it is better to be more specific, and declare only the nonzero partials.

After you declare the nonzero partial derivatives, you need to implement the compute_partials method to perform the actual derivative computations. OpenMDAO will call this method whenever it needs to work with the partial derivatives. The values are stored in the Jacobian object, J, and get used in the linear solutions that are necessary to compute model-level total derivatives. This API results in the assembly of a Jacobian matrix in memory. The compute_partials API is the most appropriate way to declare derivatives in the vast majority of circumstances, and you should use it unless you have a good reason not to.

## Providing Derivatives Using the Matrix-Free API
Sometimes you don’t want to assemble the full partial-derivative Jacobian of your component in memory. The reasons why you might not want this are beyond the scope of this tutorial. For now, let’s assume that if matrix assembly won’t work for your application, that you are likely already well aware of this issue. So if you can’t imagine why you would want to use a matrix-free API, you may disregard the following link. If you do need to work matrix-free, there is a compute_jacvec_product API, examples of which can be found in the feature document for ExplicitComponent.

## How Do I Know If My Derivatives Are Correct?
It is really important, if you are going to provide analytic derivatives, that you make sure they are correct. It is hard to overstate the importance of accurate derivatives in the convergence of analysis and optimization problems. OpenMDAO provides a helper function to make it easier to verify your partial derivatives. Any time you implement analytic derivatives, or change the nonlinear equations of your analysis, you should check your partial derivatives this way.

In [7]:
from openmdao.api import Problem, IndepVarComp

from openmdao.test_suite.test_examples.test_betz_limit import ActuatorDisc

prob = Problem()

prob.model.add_subsystem('a_disk', ActuatorDisc())

prob.setup()

prob.check_partials(compact_print=True)

---------------------
Component: 'a_disk'
---------------------
'<output>'      wrt '<variable>'    | fwd mag.   | rev mag.   | check mag. | a(fwd-chk) | a(rev-chk) | a(fwd-rev) | r(fwd-chk) | r(rev-chk) | r(fwd-rev)
--------------------------------------------------------------------------------------------------------------------------------------------------------

'Cp'            wrt 'a'             | 1.0000e+00 | 1.0000e+00 | 1.0000e+00 | 2.0000e-06 | 2.0000e-06 | 0.0000e+00 | 2.0000e-06 | 2.0000e-06 | 0.0000e+00
'Ct'            wrt 'a'             | 0.0000e+00 | 0.0000e+00 | 4.0000e-06 | 4.0000e-06 | 4.0000e-06 | 0.0000e+00 | 1.0000e+00 | 1.0000e+00 | 0.0000e+00
'Vd'            wrt 'a'             | 2.0000e+01 | 2.0000e+01 | 2.0000e+01 | 5.7511e-10 | 5.7511e-10 | 0.0000e+00 | 2.8756e-11 | 2.8756e-11 | 0.0000e+00
'Vr'            wrt 'Vu'            | 5.0000e-01 | 5.0000e-01 | 5.0000e-01 | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 | 0.0000e+00
'Vr'            w

{'a_disk': {('Cp', 'a'): {'J_fd': array([[-1.000002]]),
   'J_fwd': array([[-1.]]),
   'J_rev': array([[-1.]]),
   'abs error': ErrorTuple(forward=2.00001522898674e-06, reverse=2.00001522898674e-06, forward_reverse=0.0),
   'magnitude': MagnitudeTuple(forward=1.0, reverse=1.0, fd=1.000002000015229),
   'rel error': ErrorTuple(forward=2.000011228933824e-06, reverse=2.000011228933824e-06, forward_reverse=0.0)},
  ('Ct', 'a'): {'J_fd': array([[-4.00003046e-06]]),
   'J_fwd': array([[0.]]),
   'J_rev': array([[0.]]),
   'abs error': ErrorTuple(forward=4.00003045797348e-06, reverse=4.00003045797348e-06, forward_reverse=0.0),
   'magnitude': MagnitudeTuple(forward=0.0, reverse=0.0, fd=4.00003045797348e-06),
   'rel error': ErrorTuple(forward=1.0, reverse=1.0, forward_reverse=0.0)},
  ('Vd', 'a'): {'J_fd': array([[-20.]]),
   'J_fwd': array([[-20.]]),
   'J_rev': array([[-20.]]),
   'abs error': ErrorTuple(forward=5.751132903242251e-10, reverse=5.751132903242251e-10, forward_reverse=0.0),
   

### Running the optimization

In [6]:


# build the model
prob = Problem()
indeps = prob.model.add_subsystem('indeps', IndepVarComp(), promotes=['*'])
indeps.add_output('a', .5)
indeps.add_output('Area', 10.0, units='m**2')
indeps.add_output('rho', 1.225, units='kg/m**3')
indeps.add_output('Vu', 10.0, units='m/s')

prob.model.add_subsystem('a_disk', ActuatorDisc(),
                        promotes_inputs=['a', 'Area', 'rho', 'Vu'])

# setup the optimization
prob.driver = ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'

prob.model.add_design_var('a', lower=0., upper=1.)
prob.model.add_design_var('Area', lower=0., upper=1.)
# negative one so we maximize the objective
prob.model.add_objective('a_disk.Cp', scaler=-1)

prob.setup()
prob.run_driver()

# minimum value
print('Cp_max:', prob['a_disk.Cp'])
print('a:', prob['a'])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.5925925906659251
            Iterations: 5
            Function evaluations: 6
            Gradient evaluations: 5
Optimization Complete
-----------------------------------
Cp_max: [0.59259259]
a: [0.33335528]
